In [1]:
#!pip install ultralytics
#!apt update
#!apt install -y libgl1-mesa-glx libglib2.0-0

In [2]:
# heart_valve_detector.py
# Complete script for data preparation, training, and inference with YOLOv12
# Requirements: Run `pip install ultralytics` before executing this script
# Assumes your data is in /path/to/root (adjust ROOT_DIR below)

import os
import shutil
from ultralytics import YOLO

# Step 1: Configuration - Adjust these paths as needed
ROOT_DIR = '/NFS/celine/aicup/yoloNew'  # Root directory containing training_image, training_label, testing_image
YAML_PATH = 'data9010.yaml'
MODEL_SIZE = 'yolo12s.pt'  # Pretrained model: 'yolov12n.pt' (nano), 'yolov12s.pt' (small), etc.
#MODEL_SIZE = '/data/user/tyhuang/aicup/runs/detect/heart_valve_detector14/weights/last.pt'
EPOCHS = 400  # Number of epochs
IMG_SIZE = 512  # Image size for training (adjust to your image resolution, e.g., 512 for medical)
BATCH_SIZE = 16  # Adjust based on GPU VRAM
OUTPUT_NAME = 'heart_valve_detector_nano10'  # Name for training outputs

    
# Step 5: Load Pretrained Model
model = YOLO(MODEL_SIZE)  # Downloads if not present

# Step 6: Train the Model
print("Starting training...")
train_results = model.train(
    data=YAML_PATH,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    name=OUTPUT_NAME,
    device=0,  # GPU (change to 'cpu' if no GPU)
    patience=20,  # Early stopping
    augment=True,
    single_cls=True,  # Single class optimization
    verbose=False,
    


)
print("Training complete.")


Starting training...
New https://pypi.org/project/ultralytics/8.3.228 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.223 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3090, 24257MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data9010.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=400, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=heart_valve_detect

In [3]:
# 取得 trainer
trainer = model.trainer

# 取得訓練集影像路徑
#trainer 有屬性 train_loader而且這個 train_loader 不是 None 才會執行裡面的程式。
if hasattr(trainer, "train_loader") and trainer.train_loader is not None:
    train_imgs = trainer.train_loader.dataset.im_files
#如果 trainer 物件裡有 train_loader 這個屬性，而且它不是空的 (None)，那就使用它
else:
    raise AttributeError("trainer.train_loader 不存在，可能是訓練未成功建立或版本不相容。")

# 取得驗證集影像路徑（不同版本可能放在 val_loader 或 validator.dataloader）
if hasattr(trainer, "val_loader") and trainer.val_loader is not None:
    val_imgs = trainer.val_loader.dataset.im_files
elif hasattr(trainer, "validator") and hasattr(trainer.validator, "dataloader"):
    val_imgs = trainer.validator.dataloader.dataset.im_files
else:
    raise AttributeError("無法取得驗證集 DataLoader，請確認有設定 val 集或版本相容性。")

# 計算數量與重疊
total_train = len(train_imgs)
total_val = len(val_imgs)

# 若路徑可能不同但檔名相同，你也可以改成用檔名比對：
overlap = len(set(map(os.path.basename, train_imgs)) & set(map(os.path.basename, val_imgs)))
#overlap = len(set(train_imgs) & set(val_imgs))

print(f"訓練集影像數量: {total_train}")
print(f"驗證集影像數量: {total_val}")
print(f"重疊影像數量: {overlap}")
print(f"重疊比例: {overlap / total_val:.2%}" if total_val else "重疊比例: N/A (val 為 0)")


訓練集影像數量: 15177
驗證集影像數量: 1686
重疊影像數量: 0
重疊比例: 0.00%


In [4]:
# Step 7: Validate on Val Set
#此處是用最後一個 epoch 的模型權重（也就是 last.pt）去驗證
print("Validating model...")
val_results = model.val()
print(val_results)  # Prints metrics like mAP

Validating model...
Ultralytics 8.3.223 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3090, 24257MiB)
YOLOv12m summary (fused): 169 layers, 20,105,683 parameters, 0 gradients, 67.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 889.5±837.7 MB/s, size: 102.4 KB)
val: Scanning /NFS/celine/aicup/yoloNew/val/labels.cache... 1686 images, 1415 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1686/1686 2.0Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 106/106 5.3it/s 20.0s0.2s
                   all       1686        271      0.939      0.961      0.985      0.761
Speed: 0.5ms preprocess, 8.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /NFS/celine/aicup/yoloNew/runs/detect/val25
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix objec

這段會：

載入你剛訓練完的權重（last.pt）

讀取 data.yaml 裡指定的驗證集路徑（val）

跑推論（inference）：用模型去偵測驗證集裡的每張圖片。

比對標籤（ground truth）與預測框（predicted boxes）

計算一系列評估指標。

In [8]:
# Step 8: Inference on Test Images
#用訓練好的模型去「辨識測試影像」，並把結果（例如框框、標籤）輸出到資料夾中。

ROOT_DIR = '/NFS/celine/aicup/yoloNew'
print("Running inference on test images...")
test_path = os.path.join(ROOT_DIR, 'test_images') 
#使用 os.path.join() 組合路徑，得到完整的測試影像資料夾路徑
#例如：/NFS/celine/aicup/yoloNew/test_images


#(用last.pt推論)infer_results = model(test_path, save=True, verbose=False) # Saves detections in runs/detect/
#infer_results = model(test_path, save=True, verbose=False, save_txt=True)  

# 用 best.pt 推論
best_path = f"runs/detect/heart_valve_detector_nano1026/weights/best.pt"
best_model = YOLO(best_path)
best_infer_results = best_model(test_path, save=True, verbose=False)

print("Inference complete. Check runs/detect/ for outputs.")

Running inference on test images...
WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to /NFS/celine/aicup/yoloNew/runs/detect/predict26
Inference complete. Check runs/detect/ for outputs.


In [9]:
import os
import pandas as pd

print("Extracting bounding boxes from inference results...")

records = []

for result in best_infer_results:
#(用last.pt推論)for result in infer_results:  # 每個 result 對應一張影像
    # 去除副檔名
    img_name = os.path.splitext(os.path.basename(result.path))[0]

    boxes = result.boxes
    for box in boxes:
        xyxy = box.xyxy[0].cpu().numpy().tolist()
        conf = float(box.conf[0].cpu().numpy())
        cls = int(box.cls[0].cpu().numpy())
        # 四捨五入為整數
        x1, y1, x2, y2 = map(int, xyxy)
        records.append([img_name, cls, conf, x1, y1, x2, y2])

# 轉成 DataFrame
df = pd.DataFrame(records, columns=['image_name', 'class', 'confidence', 'x1', 'y1', 'x2', 'y2'])

# 轉成指定格式的 txt 檔
output_txt = os.path.join('runs', 'detect', OUTPUT_NAME, 'inference_boxes.txt')

with open(output_txt, 'w') as f:
    for _, row in df.iterrows():
        line = f"{row.image_name} {int(row['class'])} {row['confidence']:.4f} {int(row['x1'])} {int(row['y1'])} {int(row['x2'])} {int(row['y2'])}\n"
        f.write(line)

print(f"✅ TXT file saved to: {output_txt}")


Extracting bounding boxes from inference results...
✅ TXT file saved to: runs/detect/heart_valve_detector_nano10/inference_boxes.txt


#逐一檢查推論結果中每張圖片的偵測框信心值，並輸出每張圖片的信心值列表、最小值、最大值及差異，用來偵測異常或驗證信心分佈是否正常

import numpy as np
import torch

for result in infer_results:
    conf_t = result.boxes.conf  # tensor, shape (n,) 或 (n,1)
    if conf_t is None or conf_t.numel() == 0:
        print(result.path, "沒有任何偵測框")
        continue

    confs = conf_t.view(-1).detach().cpu().numpy()  # 轉 1D
    mn = float(confs.min())
    mx = float(confs.max())

    # 強制檢查：單框時 min==max
    if confs.size == 1 and abs(mn - mx) > 1e-9:
        print(f"[ERROR] 單一框卻出現 min!=max，confs={confs.tolist()}  mn={mn} mx={mx}")

    # 顯示完整來源，避免被其他變數污染
    print(f"{result.path} | confs={confs.tolist()} | n={confs.size} | min={mn:.6f} | max={mx:.6f} | diff={mx-mn:.6f}")


img_idx = 0  # 看第一張就好

print("=== last.pt ===")
for box in last_infer_results[img_idx].boxes:
    print("xyxy:", box.xyxy[0].cpu().numpy(), "conf:", float(box.conf[0].cpu().numpy()))

print("=== best.pt ===")
for box in best_infer_results[img_idx].boxes:
    print("xyxy:", box.xyxy[0].cpu().numpy(), "conf:", float(box.conf[0].cpu().numpy()))
